# Loop 60 Analysis - Final Day Strategy

**CRITICAL: Less than 4 hours remaining, 5 submissions left**

Goal: Beat target LB = 0.0707

In [ ]:
import numpy as np
import pandas as pd

# Submission history
submissions = [
    ('exp_000', 0.011081, 0.09816),
    ('exp_001', 0.012297, 0.10649),
    ('exp_003', 0.010501, 0.09719),
    ('exp_005', 0.010430, 0.09691),
    ('exp_006', 0.009749, 0.09457),
    ('exp_007', 0.009262, 0.09316),
    ('exp_009', 0.009192, 0.09364),
    ('exp_012', 0.009004, 0.09134),
    ('exp_024', 0.008689, 0.08929),
    ('exp_026', 0.008465, 0.08875),
    ('exp_030', 0.008298, 0.08772),
    ('exp_035', 0.009825, 0.09696),
]

df = pd.DataFrame(submissions, columns=['exp', 'cv', 'lb'])
print('=== Submission History ===')
print(df.to_string(index=False))
print(f'\nBest CV: {df.cv.min():.6f} ({df.loc[df.cv.idxmin(), "exp"]})')
print(f'Best LB: {df.lb.min():.5f} ({df.loc[df.lb.idxmin(), "exp"]})')
print(f'Target LB: 0.0707')

In [ ]:
# Linear regression to understand CV-LB relationship
from sklearn.linear_model import LinearRegression

X = df['cv'].values.reshape(-1, 1)
y = df['lb'].values

reg = LinearRegression()
reg.fit(X, y)

print(f'\n=== CV-LB Relationship ===')
print(f'LB = {reg.coef_[0]:.3f} * CV + {reg.intercept_:.4f}')
print(f'R² = {reg.score(X, y):.4f}')

# Predict LB for best CV (exp_032 = 0.008194)
best_cv = 0.008194
predicted_lb = reg.predict([[best_cv]])[0]
print(f'\nFor exp_032 (CV={best_cv}): Predicted LB = {predicted_lb:.5f}')

# What CV would we need to hit target?
target_lb = 0.0707
required_cv = (target_lb - reg.intercept_) / reg.coef_[0]
print(f'\nTo hit target LB={target_lb}: Required CV = {required_cv:.6f}')
print(f'Current best CV: {best_cv:.6f}')
print(f'Gap: {(best_cv - required_cv) / required_cv * 100:.1f}% too high')

In [ ]:
# Check residuals - which experiments beat the linear prediction?
df['predicted_lb'] = reg.predict(df[['cv']])
df['residual'] = df['lb'] - df['predicted_lb']
df['beat_prediction'] = df['residual'] < 0

print('\n=== Residual Analysis ===')
print(df.sort_values('residual')[['exp', 'cv', 'lb', 'predicted_lb', 'residual']].to_string(index=False))

print(f'\nExperiments that beat prediction: {df.beat_prediction.sum()}')
print(f'Best residual: {df.residual.min():.5f} ({df.loc[df.residual.idxmin(), "exp"]})')

In [ ]:
# Key insight: The intercept (0.052) is the floor
# Even with CV=0, we'd get LB=0.052
# Target is 0.0707, so we need to reduce intercept by 0.0707-0.052 = 0.0187

print('\n=== KEY INSIGHT ===')
print(f'Intercept (floor): {reg.intercept_:.4f}')
print(f'Target LB: 0.0707')
print(f'Gap to close: {0.0707 - reg.intercept_:.4f}')
print(f'\nThis gap represents the systematic bias between CV and LB.')
print(f'It cannot be closed by improving CV alone.')
print(f'\nPossible approaches:')
print(f'1. Find a model with different CV-LB relationship (lower intercept)')
print(f'2. Ensemble models with negative residuals')
print(f'3. Post-processing/calibration')

In [ ]:
# Strategy for remaining 5 submissions
print('\n=== SUBMISSION STRATEGY ===')
print('\nRemaining submissions: 5')
print('Time remaining: < 4 hours')
print('\nPriority order:')
print('1. Submit exp_032 (CV=0.008194) - best CV, not yet submitted')
print('   Expected LB: ~0.0872 (marginal improvement over 0.0877)')
print('\n2. If exp_032 improves LB, try variations:')
print('   - Different random seeds')
print('   - Slightly different ensemble weights')
print('\n3. If exp_032 doesn\'t improve, try:')
print('   - Ensemble of exp_030 + exp_032 predictions')
print('   - Average of best submissions')
print('\n4. Final submission: Best performing model')